In [6]:
# Core packages for word embeddings
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api

import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [7]:
import numpy as np
model_name = 'glove-twitter-25'
w2v_small = api.load(model_name)

INFO - 17:35:10: Creating /Users/kawaiyuen/gensim-data


[==================================================] 100.0% 104.8/104.8MB downloaded


INFO - 17:35:43: glove-twitter-25 downloaded
INFO - 17:35:43: loading projection weights from /Users/kawaiyuen/gensim-data/glove-twitter-25/glove-twitter-25.gz
INFO - 17:36:21: KeyedVectors lifecycle event {'msg': 'loaded (1193514, 25) matrix of type float32 from /Users/kawaiyuen/gensim-data/glove-twitter-25/glove-twitter-25.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-12-28T17:36:21.464747', 'gensim': '4.3.0', 'python': '3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [8]:
# My code
# Define word lists
target_sancvirtue = ['pure','clean','sacred','saint','modest','clean','innocent','integrity']
target_sancvice = ['disgust','disease','sin','dirt','stain','tarnish','debase','gross']
attribute_male = ['he', 'his', 'him', 'son', 'father', 'boy', 'brother', 'man', 'male']
attribute_female = ['she', 'her', 'hers', 'daughter', 'mother', 'girl', 'sister', 'woman', 'female']

# Calculate the WEAT score
def cosine_similarity(word, attribute_set):
        word_vector = w2v_small[word]
        attribute_vectors = [w2v_small[attr] for attr in attribute_set]
        return np.mean([np.dot(word_vector, attr_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(attr_vector))
                        for attr_vector in attribute_vectors])

def weat_score(word_set_X, word_set_Y, word_set_A, word_set_B):

    X_association = np.mean([cosine_similarity(x, attribute_male) - cosine_similarity(x, attribute_female)
                             for x in word_set_X])
    Y_association = np.mean([cosine_similarity(y, attribute_male) - cosine_similarity(y, attribute_female)
                             for y in word_set_Y])
    A_association = np.mean([cosine_similarity(a, attribute_male) - cosine_similarity(a, attribute_female)
                             for a in word_set_A])
    B_association = np.mean([cosine_similarity(b, attribute_male) - cosine_similarity(b, attribute_female)
                             for b in word_set_B])

    return X_association - Y_association / np.sqrt(np.var([X_association, Y_association]) +
                                                    np.var([A_association, B_association]))

# Calculate the WEAT score for the given word sets
weat_result = weat_score(target_sancvirtue, target_sancvice, attribute_male, attribute_female)

# Step 4: Print the WEAT result
print(f"WEAT score: {weat_result}")

WEAT score: 1.05934739112854
